In [1]:
import sys
sys.path.append("../")
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import os
import argparse
import random
import tqdm
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.metrics import ClassificationMetric
from sklearn.preprocessing import StandardScaler

from aif360.datasets import AdultDataset, GermanDataset, CompasDataset
from utility import get_data,write_to_file
from sklearn.base import clone
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
import copy
from sklearn.tree._tree import TREE_LEAF
import tqdm

In [6]:
dataset_used = "meps19" # "adult", "german", "compas" ,"meps19"
attr = "race"
trials = 50
dataset_orig, privileged_groups,unprivileged_groups,optim_options = get_data(dataset_used, attr)

In [3]:
def get_metrics(clf,test,test_pred,unprivileged_groups,privileged_groups):
    pred = clf.predict(test.features).reshape(-1,1)

    #dataset_orig_test_pred = test.copy(deepcopy=True)
    test_pred.labels = pred
    class_metric = ClassificationMetric(test, test_pred,
                         unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
    stat = abs(class_metric.statistical_parity_difference())
    aod = abs(class_metric.average_abs_odds_difference())
    disp = abs(1-class_metric.disparate_impact())
    eod = abs(class_metric.equal_opportunity_difference())
    acc = class_metric.accuracy()
    return acc,stat,disp,aod,eod

In [7]:
data = []
for t in tqdm.tqdm(range(trials)):
    np.random.seed(t)
    dataset_orig_train, dataset_orig_test = dataset_orig.split([0.7], shuffle=True)
    dataset_orig_test,dataset_orig_valid = dataset_orig_test.split([0.5], shuffle=True)
    dataset_orig_test_pred = dataset_orig_test.copy(deepcopy=True)
    dataset_orig_train_pred = dataset_orig_train.copy(deepcopy=True)
    dataset_orig_valid_pred = dataset_orig_valid.copy(deepcopy=True)
    clf = GradientBoostingClassifier(random_state=1)
    clf = clf.fit(dataset_orig_train.features, dataset_orig_train.labels)
    train_acc,train_stat,train_disp,train_aod,train_eod = get_metrics(clf,dataset_orig_train,dataset_orig_train_pred,unprivileged_groups,privileged_groups)
    test_acc,test_stat,test_disp,test_aod,test_eod = get_metrics(clf,dataset_orig_test,dataset_orig_test_pred,unprivileged_groups,privileged_groups)
    valid_acc,valid_stat,valid_disp,valid_aod,valid_eod = get_metrics(clf,dataset_orig_valid,dataset_orig_valid_pred,unprivileged_groups,privileged_groups)
    data.append([test_acc,test_stat,test_disp,test_aod,test_eod])
data = np.array(data)
np.mean(data,axis = 0)

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [05:09<00:00,  6.19s/it]


array([0.83831022, 0.12330756, 0.64563894, 0.10900563, 0.16761354])

In [8]:
for row in data:
    print (*row)

0.8326608432620403 0.06960982356391737 0.4718741808723461 0.028541326956325143 0.019897946569812364
0.8371991285614381 0.1370066972475993 0.7022076455479667 0.13165983589768182 0.19449911095001715
0.8366732847700943 0.12356671109017289 0.6609353190978793 0.1192984124109927 0.19002841646031537
0.8354024303479668 0.1125244718084322 0.579340811397663 0.11665906088056513 0.2038281865927805
0.8328693433486604 0.1281651555055861 0.6412183574265267 0.10007112323887041 0.14558195208969915
0.8465861883320193 0.1220983878152324 0.6586066679613023 0.1235152286140643 0.19272042367041997
0.8504662307925062 0.12638220505875006 0.6382673168049229 0.12133073592941435 0.20723083636737671
0.8375845292179679 0.10523184064540356 0.6061536579415211 0.08647123847764915 0.139456574892513
0.8420930660750732 0.1224468766961717 0.6588433406881831 0.13957833363225086 0.21924114228783986
0.8513399464132557 0.10878165860240589 0.6022309309250682 0.10650108034023206 0.17808386823816663
0.8431739412556495 0.10845446

In [5]:
np.std(data,axis = 0)

array([0.0091155 , 0.01691314, 0.06547371, 0.02788286, 0.05333622])